In [2]:
import os
import pandas as pd
import librosa
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from PIL import Image # For image processing

# Keras/TensorFlow 관련 라이브러리
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50, EfficientNetB0 # ResNet, EfficientNet 사전학습 모델

# pydub 라이브러리 (MP3 to WAV 변환용)
from pydub import AudioSegment

# 새로운 데이터셋 기본 경로 설정 (MP3 파일이 있는 곳)
base_data_path = r'C:\Users\jh\Documents\GitHub\BearingGuardian\Dataset\bearing-raw-mp3'

# 스펙트로그램 이미지를 저장할 경로 설정
output_spectrogram_dir = os.path.join(base_data_path, 'spectrograms')

# 필요한 하위 디렉토리 미리 생성
os.makedirs(os.path.join(output_spectrogram_dir, 'train', 'normal'), exist_ok=True)
os.makedirs(os.path.join(output_spectrogram_dir, 'test', 'normal'), exist_ok=True)
os.makedirs(os.path.join(output_spectrogram_dir, 'test', 'anomaly'), exist_ok=True)
os.makedirs(os.path.join(output_spectrogram_dir, 'val', 'normal'), exist_ok=True) # Keras ImageDataGenerator용
os.makedirs(os.path.join(output_spectrogram_dir, 'val', 'anomaly'), exist_ok=True) # Keras ImageDataGenerator용

print(f"설정된 기본 데이터 경로: {base_data_path}")
print(f"설정된 스펙트로그램 저장 경로: {output_spectrogram_dir}")


설정된 기본 데이터 경로: C:\Users\jh\Documents\GitHub\BearingGuardian\Dataset\bearing-raw-mp3
설정된 스펙트로그램 저장 경로: C:\Users\jh\Documents\GitHub\BearingGuardian\Dataset\bearing-raw-mp3\spectrograms


In [3]:
# 'attributes_00.csv' 파일은 'base_data_path'의 상위 폴더인 'Dataset' 아래 'bearing' 폴더에 있을 수 있습니다.
# 사용자께서 이전에 '/Users/pjh_air/Documents/SJ_simhwa/final/dcase/bearing/attributes_00.csv'에서 로드했습니다.
# 따라서, 현재 'base_data_path'는 'bearing-raw-mp3'이므로, 'bearing' 폴더 경로를 명확히 해야 합니다.

# DCASE 데이터셋의 'bearing' 폴더 경로를 가정합니다.
# 'base_data_path'와 같은 레벨에 'bearing' 폴더가 있을 수 있습니다.
# 예를 들어, 'C:\Users\jh\Documents\GitHub\BearingGuardian\Dataset\bearing\attributes_00.csv'
dcase_bearing_folder_path = os.path.join(os.path.dirname(base_data_path), 'bearing-raw-mp3')
meta_file_path = os.path.join(dcase_bearing_folder_path, 'attributes_00.csv')

try:
    df_meta_original = pd.read_csv(meta_file_path)

    # 기존 file_name 컬럼에서 'bearing/' 접두사 제거 (WAV 파일 경로 맞추기 위함)
    # df_meta_original['file_name'] 컬럼의 값이 'bearing/train/...' 형태라고 가정
    df_meta_original['relative_wav_path'] = df_meta_original['file_name'].apply(lambda x: x.replace('bearing/', '', 1))
    
    # parsed_label, parsed_subset 등 파싱된 정보도 그대로 사용
    df_meta_original['parsed_label'] = df_meta_original['relative_wav_path'].apply(lambda x: 'normal' if 'normal' in x else ('anomaly' if 'anomaly' in x else 'unknown'))
    df_meta_original['parsed_subset'] = df_meta_original['relative_wav_path'].apply(lambda x: 'train' if 'train' in x else ('test' if 'test' in x else 'unknown'))
    df_meta_original['machine_id'] = 'bearing' # DCASE bearing 데이터이므로 고정

    df_meta = df_meta_original # 최종적으로 사용할 메타데이터 DataFrame

    print("\n--- WAV 기반 메타데이터 (df_meta.head()) ---")
    print(df_meta.head())
    print(f"\n총 파일 수: {len(df_meta)}")
    print(f"\n'parsed_label' (정상/이상)별 데이터 개수:\n{df_meta['parsed_label'].value_counts()}")
    print(f"\n'parsed_subset' (훈련/테스트)별 데이터 개수:\n{df_meta['parsed_subset'].value_counts()}")
    print(f"\n'parsed_subset'와 'parsed_label' 조합별 데이터 개수:\n{df_meta.groupby(['parsed_subset', 'parsed_label']).size().unstack(fill_value=0)}")

except FileNotFoundError:
    print(f"오류: 원본 메타데이터 파일을 찾을 수 없습니다. 경로를 다시 확인해주세요: {meta_file_path}")
    print("attributes_00.csv 파일이 올바른 위치에 있는지 확인해주세요.")
except Exception as e:
    print(f"WAV 기반 메타데이터 로드 및 파싱 중 오류 발생: {e}")
    exit()


--- WAV 기반 메타데이터 (df_meta.head()) ---
                                           file_name  \
0  bearing/test/section_00_source_test_anomaly_00...   
1  bearing/test/section_00_source_test_anomaly_00...   
2  bearing/test/section_00_source_test_anomaly_00...   
3  bearing/test/section_00_source_test_anomaly_00...   
4  bearing/test/section_00_source_test_anomaly_00...   

                                   relative_wav_path parsed_label  \
0  test/section_00_source_test_anomaly_0000_noAtt...      anomaly   
1  test/section_00_source_test_anomaly_0001_noAtt...      anomaly   
2  test/section_00_source_test_anomaly_0002_noAtt...      anomaly   
3  test/section_00_source_test_anomaly_0003_noAtt...      anomaly   
4  test/section_00_source_test_anomaly_0004_noAtt...      anomaly   

  parsed_subset machine_id  
0          test    bearing  
1          test    bearing  
2          test    bearing  
3          test    bearing  
4          test    bearing  

총 파일 수: 1200

'parsed_label' (정상/이

In [8]:
# 스펙트로그램 생성 파라미터
sr_target = 16000
n_fft = 1024
hop_length = 512
n_mels = 128

print(f"\n스펙트로그램 저장 경로: {output_spectrogram_dir}")

def create_and_save_spectrogram_from_wav(row):
    try:
        # WAV 파일의 실제 경로 구성
        # 'base_data_path' (C:\Users\...\Dataset\bearing-raw-mp3)와
        # 'row['relative_wav_path']' (test/section_00_...wav)를 결합합니다.
        # 'relative_wav_path'는 이미 'bearing/' 접두사가 제거된 상태입니다.
        full_wav_file_path = os.path.join(base_data_path, row['relative_wav_path'])

        # 디버깅을 위해 로드하려는 최종 경로 출력 (필요시 주석 해제)
        # print(f"DEBUG: Attempting to load WAV from: {full_wav_file_path}")
        # if not os.path.exists(full_wav_file_path):
        #     print(f"DEBUG: WAV file NOT FOUND at: {full_wav_file_path}")
        #     raise FileNotFoundError(f"WAV file not found: {full_wav_file_path}")

        # librosa.load: 오디오 파일 로드. sr_target으로 리샘플링됩니다.
        y, sr = librosa.load(full_wav_file_path, sr=sr_target)

        # Mel 스펙트로그램 생성
        mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
        # dB 스케일로 변환 (이미지 시각화 및 딥러닝 입력에 적합)
        S_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

        # 스펙트로그램 이미지 저장 경로 구성: output_spectrogram_dir / subset / label / base_filename_png
        
        # 순수한 파일명 추출 (확장자 포함). file_name 컬럼은 'bearing/test/section_00...wav' 형태
        base_filename_wav = os.path.basename(os.path.normpath(row['file_name'])) 
        base_filename_png = base_filename_wav.replace('.wav', '.png') # '.png'로 변경
        
        subset = row['parsed_subset'] # 'train' 또는 'test'
        label = row['parsed_label']   # 'normal' 또는 'anomaly'
        
        save_path = os.path.join(output_spectrogram_dir, subset, label, base_filename_png)
        
        # 저장할 상위 디렉토리가 없으면 생성 (예: spectrograms_from_wav/train/normal)
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        # Matplotlib을 사용하여 스펙트로그램 이미지를 생성하고 저장
        # 축, 여백 없이 순수 이미지 데이터만 저장하여 딥러닝 모델 입력에 적합하도록 함
        fig = plt.figure(figsize=(S_db.shape[1]/100, S_db.shape[0]/100), dpi=100) # 이미지 해상도 조절
        ax = plt.Axes(fig, [0., 0., 1., 1.]) # figure 전체를 차지하는 축 생성
        ax.set_axis_off() # 축 제거
        fig.add_axes(ax) # 축을 figure에 추가
        
        ax.imshow(S_db, origin='lower', aspect='auto', cmap=cm.magma) # 이미지 데이터 플로팅
        plt.savefig(save_path) # 파일로 저장
        plt.close(fig) # 메모리 누수 방지

        return True
    except Exception as e:
        print(f"오류: '{row['file_name']}' 파일 처리 중 오류 발생: {e}")
        return False

print("\n--- WAV 기반 스펙트로그램 생성 시작 ---")
# df_meta DataFrame을 사용하여 모든 파일에 대해 스펙트로그램 생성
for idx, row in tqdm(df_meta.iterrows(), total=len(df_meta), desc="Generating Spectrograms"):
    create_and_save_spectrogram_from_wav(row)

print("--- WAV 기반 스펙트로그램 생성 완료 ---")



스펙트로그램 저장 경로: C:\Users\jh\Documents\GitHub\BearingGuardian\Dataset\bearing-raw-mp3\spectrograms

--- WAV 기반 스펙트로그램 생성 시작 ---


Generating Spectrograms: 100%|██████████| 1200/1200 [00:32<00:00, 36.70it/s]

--- WAV 기반 스펙트로그램 생성 완료 ---


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50, EfficientNetB0 # ResNet, EfficientNet 사전학습 모델

# 이전 단계에서 설정된 이미지 크기 및 클래스 수 변수 사용:
# img_height, img_width = n_mels, int(sr_target * 10 / hop_length) 
# input_shape = (img_height, img_width, 3) 
# num_classes = 2 # normal, anomaly

def build_keras_model(model_name, input_shape, num_classes=2):
    model = Sequential()

    if model_name == 'custom_cnn':
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5)) # 과적합 방지: 훈련 시 50%의 뉴런을 무작위로 비활성화
        model.add(Dense(num_classes, activation='sigmoid' if num_classes == 2 else 'softmax')) 

    elif model_name == 'resnet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
        model.add(base_model)
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5)) # 과적합 방지
        model.add(Dense(num_classes, activation='sigmoid' if num_classes == 2 else 'softmax'))
        
        base_model.trainable = False # 사전 학습된 레이어는 훈련 동결 (전이 학습)

    elif model_name == 'efficientnetb0':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
        model.add(base_model)
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5)) # 과적합 방지
        model.add(Dense(num_classes, activation='sigmoid' if num_classes == 2 else 'softmax'))
        
        base_model.trainable = False # 사전 학습된 레이어는 훈련 동결
    else:
        raise ValueError(f"Unknown model name: {model_name}")
    
    return model


In [12]:
# 이전 단계에서 설정된 train_generator, validation_generator, test_generator 사용

def train_and_evaluate_keras_model(model, train_gen, val_gen, test_gen, model_name, epochs=10):
    print(f"\n--- {model_name} 학습 시작 ---")
    
    # 모델 컴파일
    model.compile(optimizer=Adam(learning_rate=0.001), # Adam 옵티마이저, 학습률 0.001
                  loss='binary_crossentropy' if num_classes == 2 else 'categorical_crossentropy', # 이진 분류 손실 함수
                  metrics=['accuracy']) # 정확도 지표 사용
    
    # 모델 구조 요약 출력
    model.summary()

    # 콜백 (Early Stopping): 과적합 방지를 위해 사용
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
    
    # 모델 학습
    history = model.fit(
        train_gen, # 훈련 데이터 제너레이터
        epochs=epochs, # 학습 에포크 수
        validation_data=val_gen, # 검증 데이터 제너레이터
        callbacks=callbacks # 콜백 적용
    )
    
    print(f"\n--- {model_name} 학습 완료 ---")

    # 모델 평가 (테스트 셋)
    print(f"\n--- {model_name} 테스트 셋 평가 ---")
    loss, accuracy = model.evaluate(test_gen) # 테스트 셋으로 손실과 정확도 계산
    print(f'Test Loss: {loss:.4f}')
    print(f'Test Accuracy: {accuracy:.4f}')
    
    return model, history # 학습된 모델과 학습 이력 반환


In [16]:
# # 이전 단계에서 설정된 이미지 크기 및 클래스 수 변수들:
# img_height, img_width #(스펙트로그램 높이, 너비)
# input_shape (img_height, img_width, 3)
# num_classes #(현재 2: normal, anomaly)

# 에포크 수 설정 (실제 학습 시 더 높게 설정할 수 있음)
epochs_to_train = 10 

# ===== Custom CNN 모델 학습 시작 =====
print("===== Custom CNN 모델 학습 시작 =====")
custom_cnn_model = build_keras_model('custom_cnn', input_shape, num_classes)
trained_custom_cnn_model, history_cnn = train_and_evaluate_keras_model(
    custom_cnn_model, train_generator, validation_generator, test_generator, "Custom CNN", epochs=epochs_to_train
)

# ===== ResNet50 모델 학습 시작 =====
print("\n===== ResNet50 모델 학습 시작 =====")
resnet50_model = build_keras_model('resnet50', input_shape, num_classes)
trained_resnet50_model, history_resnet50 = train_and_evaluate_keras_model(
    resnet50_model, train_generator, validation_generator, test_generator, "ResNet50", epochs=epochs_to_train
)

# ===== EfficientNetB0 모델 학습 시작 =====
print("\n===== EfficientNetB0 모델 학습 시작 =====")
efficientnetb0_model = build_keras_model('efficientnetb0', input_shape, num_classes)
trained_efficientnetb0_model, history_efficientnetb0 = train_and_evaluate_keras_model(
    efficientnetb0_model, train_generator, validation_generator, test_generator, "EfficientNetB0", epochs=epochs_to_train
)

print("\n모든 Keras 모델 학습 완료!")

===== Custom CNN 모델 학습 시작 =====


NameError: name 'input_shape' is not defined

In [3]:
import os
import pandas as pd
import librosa
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from PIL import Image

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50, EfficientNetB0


# --- 1. 경로 및 기본 설정 (이전과 동일) ---
base_data_path = r'C:\Users\jh\Documents\GitHub\BearingGuardian\Dataset\bearing-raw-mp3'
output_spectrogram_dir = os.path.join(os.path.dirname(base_data_path), 'spectrograms_from_wav')

os.makedirs(os.path.join(output_spectrogram_dir, 'train', 'normal'), exist_ok=True)
os.makedirs(os.path.join(output_spectrogram_dir, 'train', 'anomaly'), exist_ok=True)
os.makedirs(os.path.join(output_spectrogram_dir, 'val', 'normal'), exist_ok=True)
os.makedirs(os.path.join(output_spectrogram_dir, 'val', 'anomaly'), exist_ok=True)
os.makedirs(os.path.join(output_spectrogram_dir, 'test', 'normal'), exist_ok=True)
os.makedirs(os.path.join(output_spectrogram_dir, 'test', 'anomaly'), exist_ok=True)


print(f"설정된 기본 데이터 경로 (원본 WAV): {base_data_path}")
print(f"설정된 스펙트로그램 저장 경로: {output_spectrogram_dir}")

# --- 2. 메타데이터 로드 및 파싱 (이전과 동일) ---
meta_file_path = os.path.join(os.path.dirname(base_data_path), 'bearing', 'attributes_00.csv')

try:
    df_meta_original = pd.read_csv(meta_file_path)
    df_meta_original['relative_wav_path'] = df_meta_original['file_name'].apply(lambda x: x.replace('bearing/', '', 1))
    df_meta_original['parsed_label'] = df_meta_original['relative_wav_path'].apply(lambda x: 'normal' if 'normal' in x else ('anomaly' if 'anomaly' in x else 'unknown'))
    df_meta_original['parsed_subset'] = df_meta_original['relative_wav_path'].apply(lambda x: 'train' if 'train' in x else ('test' if 'test' in x else 'unknown'))
    df_meta_original['machine_id'] = 'bearing'
    df_meta = df_meta_original
    
    print("\n--- WAV 기반 메타데이터 (df_meta.head()) ---")
    print(df_meta.head())
    print(f"\n총 파일 수: {len(df_meta)}")
    print(f"\n'parsed_label' (정상/이상)별 데이터 개수:\n{df_meta['parsed_label'].value_counts()}")
    print(f"\n'parsed_subset' (훈련/테스트)별 데이터 개수:\n{df_meta['parsed_subset'].value_counts()}")
    print(f"\n'parsed_subset'와 'parsed_label' 조합별 데이터 개수:\n{df_meta.groupby(['parsed_subset', 'parsed_label']).size().unstack(fill_value=0)}")

except FileNotFoundError:
    print(f"오류: 원본 메타데이터 파일을 찾을 수 없습니다. 경로를 다시 확인해주세요: {meta_file_path}")
    exit()
except Exception as e:
    print(f"WAV 기반 메타데이터 로드 및 파싱 중 오류 발생: {e}")
    exit()


# --- 3. 스펙트로그램 생성 및 저장 (이전과 동일) ---
sr_target = 16000
n_fft = 1024
hop_length = 512
n_mels = 128

print(f"\n스펙트로그램 저장 경로: {output_spectrogram_dir}")

def create_and_save_spectrogram_from_wav(row):
    try:
        full_wav_file_path = os.path.join(base_data_path, row['relative_wav_path'])
        y, sr = librosa.load(full_wav_file_path, sr=sr_target)
        mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
        S_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

        base_filename_wav = os.path.basename(os.path.normpath(row['file_name'])) 
        base_filename_png = base_filename_wav.replace('.wav', '.png')
        
        subset = row['parsed_subset']
        label = row['parsed_label']
        
        save_path = os.path.join(output_spectrogram_dir, subset, label, base_filename_png)
        
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        fig = plt.figure(figsize=(S_db.shape[1]/100, S_db.shape[0]/100), dpi=100)
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)
        
        ax.imshow(S_db, origin='lower', aspect='auto', cmap=cm.magma) 
        plt.savefig(save_path)
        plt.close(fig) 

        return True
    except Exception as e:
        print(f"오류: '{row['file_name']}' 파일 처리 중 오류 발생: {e}")
        return False

print("\n--- WAV 기반 스펙트로그램 생성 시작 ---")
for idx, row in tqdm(df_meta.iterrows(), total=len(df_meta), desc="Generating Spectrograms"):
    create_and_save_spectrogram_from_wav(row)

print("--- WAV 기반 스펙트로그램 생성 완료 ---")

# --- 4. Keras 모델 선정 및 구축: ImageDataGenerator 설정 및 데이터 분할 수정 (이전과 동일) ---
img_height, img_width = n_mels, int(sr_target * 10 / hop_length) 
input_shape = (img_height, img_width, 3) 

batch_size = 32
num_classes = 2 # normal, anomaly (이진 분류)

train_val_df = df_meta[df_meta['parsed_subset'] == 'train'].copy()
test_df_keras = df_meta[df_meta['parsed_subset'] == 'test'].copy()

train_df_keras, val_df_keras = train_test_split(train_val_df, test_size=0.2, random_state=42)

def get_spectrogram_filepath_for_keras(row, base_spectrogram_dir):
    base_filename_wav = os.path.basename(os.path.normpath(row['file_name']))
    base_filename_png = base_filename_wav.replace('.wav', '.png')
    
    subset_folder = row['parsed_subset']
    label_folder = row['parsed_label']
    
    return os.path.join(base_spectrogram_dir, subset_folder, label_folder, base_filename_png)

train_df_keras['filepath'] = train_df_keras.apply(lambda row: get_spectrogram_filepath_for_keras(row, output_spectrogram_dir), axis=1)
train_df_keras['class'] = train_df_keras['parsed_label']

val_df_keras['filepath'] = val_df_keras.apply(lambda row: get_spectrogram_filepath_for_keras(row, output_spectrogram_dir), axis=1)
val_df_keras['class'] = val_df_keras['parsed_label']

test_df_keras['filepath'] = test_df_keras.apply(lambda row: get_spectrogram_filepath_for_keras(row, output_spectrogram_dir), axis=1)
test_df_keras['class'] = test_df_keras['parsed_label']


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

keras_class_names = ['anomaly', 'normal'] # 클래스 순서 (알파벳 순서에 따름)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df_keras,
    x_col='filepath',
    y_col='class',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary', # 이진 분류 (출력 뉴런 1개)
    classes=keras_class_names, # 클래스 라벨 명시
    shuffle=True
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df_keras,
    x_col='filepath',
    y_col='class',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    classes=keras_class_names,
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df_keras,
    x_col='filepath',
    y_col='class',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    classes=keras_class_names,
    shuffle=False
)

print("\n--- ImageDataGenerator 설정 완료 ---")
print(f"훈련 제너레이터 클래스 인덱스: {train_generator.class_indices}")
print(f"검증 제너레이터 클래스 인덱스: {validation_generator.class_indices}")
print(f"테스트 제너레이터 클래스 인덱스: {test_generator.class_indices}")


# --- 5. Keras 모델 구축 및 학습 ---
def build_keras_model(model_name, input_shape, num_classes): # num_classes는 1 (이진 분류)
    model = Sequential()

    if model_name == 'custom_cnn':
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid')) # 🐞 핵심 수정: 이진 분류이므로 출력 뉴런 1개, sigmoid 활성화 함수

    elif model_name == 'resnet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
        model.add(base_model)
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid')) # 🐞 핵심 수정

        base_model.trainable = False

    elif model_name == 'efficientnetb0':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
        model.add(base_model)
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid')) # 🐞 핵심 수정

        base_model.trainable = False
    else:
        raise ValueError(f"Unknown model name: {model_name}")
    
    return model

def train_and_evaluate_keras_model(model, train_gen, val_gen, test_gen, model_name, epochs=10):
    print(f"\n--- {model_name} 학습 시작 ---")
    
    # 모델 컴파일
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy', # 이진 분류 손실 함수
                  metrics=['accuracy'])
    
    model.summary()

    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
    
    history = model.fit(
        train_gen,
        epochs=epochs,
        validation_data=val_gen,
        callbacks=callbacks
    )
    
    print(f"\n--- {model_name} 학습 완료 ---")

    print(f"\n--- {model_name} 테스트 셋 평가 ---")
    loss, accuracy = model.evaluate(test_gen)
    print(f'Test Loss: {loss:.4f}')
    print(f'Test Accuracy: {accuracy:.4f}')
    
    return model, history

epochs_to_train = 10

print("===== Custom CNN 모델 학습 시작 =====")
# build_keras_model 호출 시 num_classes를 1로 전달
custom_cnn_model = build_keras_model('custom_cnn', input_shape, 1) 
trained_custom_cnn_model, history_cnn = train_and_evaluate_keras_model(
    custom_cnn_model, train_generator, validation_generator, test_generator, "Custom CNN", epochs=epochs_to_train
)

print("\n===== ResNet50 모델 학습 시작 =====")
# build_keras_model 호출 시 num_classes를 1로 전달
resnet50_model = build_keras_model('resnet50', input_shape, 1) 
trained_resnet50_model, history_resnet50 = train_and_evaluate_keras_model(
    resnet50_model, train_generator, validation_generator, test_generator, "ResNet50", epochs=epochs_to_train
)

print("\n===== EfficientNetB0 모델 학습 시작 =====")
# build_keras_model 호출 시 num_classes를 1로 전달
efficientnetb0_model = build_keras_model('efficientnetb0', input_shape, 1) 
trained_efficientnetb0_model, history_efficientnetb0 = train_and_evaluate_keras_model(
    efficientnetb0_model, train_generator, validation_generator, test_generator, "EfficientNetB0", epochs=epochs_to_train
)

print("\n모든 Keras 모델 학습 완료!")


설정된 기본 데이터 경로 (원본 WAV): C:\Users\jh\Documents\GitHub\BearingGuardian\Dataset\bearing-raw-mp3
설정된 스펙트로그램 저장 경로: C:\Users\jh\Documents\GitHub\BearingGuardian\Dataset\spectrograms_from_wav

--- WAV 기반 메타데이터 (df_meta.head()) ---
                                           file_name  \
0  bearing/test/section_00_source_test_anomaly_00...   
1  bearing/test/section_00_source_test_anomaly_00...   
2  bearing/test/section_00_source_test_anomaly_00...   
3  bearing/test/section_00_source_test_anomaly_00...   
4  bearing/test/section_00_source_test_anomaly_00...   

                                   relative_wav_path parsed_label  \
0  test/section_00_source_test_anomaly_0000_noAtt...      anomaly   
1  test/section_00_source_test_anomaly_0001_noAtt...      anomaly   
2  test/section_00_source_test_anomaly_0002_noAtt...      anomaly   
3  test/section_00_source_test_anomaly_0003_noAtt...      anomaly   
4  test/section_00_source_test_anomaly_0004_noAtt...      anomaly   

  parsed_subset machine

Generating Spectrograms: 100%|██████████| 1200/1200 [00:31<00:00, 38.39it/s]

--- WAV 기반 스펙트로그램 생성 완료 ---
Found 800 validated image filenames belonging to 2 classes.


Found 200 validated image filenames belonging to 2 classes.
Found 200 validated image filenames belonging to 2 classes.

--- ImageDataGenerator 설정 완료 ---
훈련 제너레이터 클래스 인덱스: {'anomaly': 0, 'normal': 1}
검증 제너레이터 클래스 인덱스: {'anomaly': 0, 'normal': 1}
테스트 제너레이터 클래스 인덱스: {'anomaly': 0, 'normal': 1}
===== Custom CNN 모델 학습 시작 =====

--- Custom CNN 학습 시작 ---


c:\Users\jh\.conda\envs\tf-gpu\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 4, 10, 1280)    │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 51200)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     6,553,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,603,428 (40.45 MB)

 Trainable params: 6,553,857 (25.00 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 547ms/step - accuracy: 0.8888 - loss: 0.1401 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 12s 475ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 12s 476ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 12s 479ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00

--- EfficientNetB0 학습 완료 ---

--- EfficientNetB0 테스트 셋 평가 ---
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 216ms/step - accuracy: 0.3678 - loss: 168.4320
Test Loss: 133.2191
Test Accuracy: 0.5000

모든 Keras 모델 학습 완료!


In [4]:
print(f"\n최종 훈련 세트 라벨 분포:\n{train_df_final['parsed_label'].value_counts()}")
print(f"최종 검증 세트 라벨 분포:\n{val_df_final['parsed_label'].value_counts()}")
print(f"최종 테스트 세트 라벨 분포:\n{test_df_final['parsed_label'].value_counts()}")


최종 훈련 세트 라벨 분포:
parsed_label
normal     660
anomaly     60
Name: count, dtype: int64
최종 검증 세트 라벨 분포:
parsed_label
normal     220
anomaly     20
Name: count, dtype: int64
최종 테스트 세트 라벨 분포:
parsed_label
normal     220
anomaly     20
Name: count, dtype: int64


In [7]:
import os
import pandas as pd
import librosa
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from PIL import Image

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score # 평가 지표 추가

# --- 1. 경로 및 기본 설정 (이전과 동일) ---
base_data_path = r'C:\Users\jh\Documents\GitHub\BearingGuardian\Dataset\bearing-raw-mp3'
output_spectrogram_dir = os.path.join(os.path.dirname(base_data_path), 'spectrograms_from_wav')

os.makedirs(os.path.join(output_spectrogram_dir, 'train', 'normal'), exist_ok=True)
os.makedirs(os.path.join(output_spectrogram_dir, 'train', 'anomaly'), exist_ok=True)
os.makedirs(os.path.join(output_spectrogram_dir, 'val', 'normal'), exist_ok=True)
os.makedirs(os.path.join(output_spectrogram_dir, 'val', 'anomaly'), exist_ok=True)
os.makedirs(os.path.join(output_spectrogram_dir, 'test', 'normal'), exist_ok=True)
os.makedirs(os.path.join(output_spectrogram_dir, 'test', 'anomaly'), exist_ok=True)

print(f"설정된 기본 데이터 경로 (원본 WAV): {base_data_path}")
print(f"설정된 스펙트로그램 저장 경로: {output_spectrogram_dir}")

# --- 2. 메타데이터 로드 및 파싱 (이전과 동일) ---
meta_file_path = os.path.join(os.path.dirname(base_data_path), 'bearing', 'attributes_00.csv')

try:
    df_meta_original = pd.read_csv(meta_file_path)
    df_meta_original['relative_wav_path'] = df_meta_original['file_name'].apply(lambda x: x.replace('bearing/', '', 1))
    df_meta_original['parsed_label'] = df_meta_original['relative_wav_path'].apply(lambda x: 'normal' if 'normal' in x else ('anomaly' if 'anomaly' in x else 'unknown'))
    df_meta_original['parsed_subset'] = df_meta_original['relative_wav_path'].apply(lambda x: 'train' if 'train' in x else ('test' if 'test' in x else 'unknown'))
    df_meta_original['machine_id'] = 'bearing'
    df_meta = df_meta_original
    
    print("\n--- WAV 기반 메타데이터 (df_meta.head()) ---")
    print(df_meta.head())
    print(f"\n총 파일 수: {len(df_meta)}")
    print(f"\n'parsed_label' (정상/이상)별 데이터 개수:\n{df_meta['parsed_label'].value_counts()}")
    print(f"\n'parsed_subset' (훈련/테스트)별 데이터 개수:\n{df_meta['parsed_subset'].value_counts()}")
    print(f"\n'parsed_subset'와 'parsed_label' 조합별 데이터 개수:\n{df_meta.groupby(['parsed_subset', 'parsed_label']).size().unstack(fill_value=0)}")

except FileNotFoundError:
    print(f"오류: 원본 메타데이터 파일을 찾을 수 없습니다. 경로를 다시 확인해주세요: {meta_file_path}")
    exit()
except Exception as e:
    print(f"WAV 기반 메타데이터 로드 및 파싱 중 오류 발생: {e}")
    exit()


# --- 3. 스펙트로그램 생성 및 저장 (이전과 동일) ---
sr_target = 16000
n_fft = 1024
hop_length = 512
n_mels = 128

print(f"\n스펙트로그램 저장 경로: {output_spectrogram_dir}")

def create_and_save_spectrogram_from_wav(row):
    try:
        full_wav_file_path = os.path.join(base_data_path, row['relative_wav_path'])
        y, sr = librosa.load(full_wav_file_path, sr=sr_target)
        mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
        S_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

        base_filename_wav = os.path.basename(os.path.normpath(row['file_name'])) 
        base_filename_png = base_filename_wav.replace('.wav', '.png')
        
        subset = row['parsed_subset']
        label = row['parsed_label']
        
        save_path = os.path.join(output_spectrogram_dir, subset, label, base_filename_png)
        
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        fig = plt.figure(figsize=(S_db.shape[1]/100, S_db.shape[0]/100), dpi=100)
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)
        
        ax.imshow(S_db, origin='lower', aspect='auto', cmap=cm.magma) 
        plt.savefig(save_path)
        plt.close(fig) 

        return True
    except Exception as e:
        print(f"오류: '{row['file_name']}' 파일 처리 중 오류 발생: {e}")
        return False

print("\n--- WAV 기반 스펙트로그램 생성 시작 ---")
for idx, row in tqdm(df_meta.iterrows(), total=len(df_meta), desc="Generating Spectrograms"):
    create_and_save_spectrogram_from_wav(row)

print("--- WAV 기반 스펙트로그램 생성 완료 ---")

# --- 4. Keras 모델 선정 및 구축: ImageDataGenerator 설정 및 데이터 분할 ---
img_height, img_width = n_mels, int(sr_target * 10 / hop_length) 
input_shape = (img_height, img_width, 3) 

batch_size = 32
num_classes = 2 # normal, anomaly (이진 분류)

# 전체 데이터 (df_meta)를 훈련/검증/테스트로 재분할
train_val_combined_df, test_df_keras = train_test_split(
    df_meta, test_size=0.2, random_state=42, stratify=df_meta['parsed_label']
)
train_df_keras, val_df_keras = train_test_split(
    train_val_combined_df, test_size=0.25, random_state=42, stratify=train_val_combined_df['parsed_label']
)

# 🐞 최종 훈련/검증/테스트 세트 라벨 분포 재확인 및 출력
print(f"\n새로운 데이터 분할 결과:")
print(f"  최종 훈련 세트 크기: {len(train_df_keras)}")
print(f"  최종 검증 세트 크기: {len(val_df_keras)}")
print(f"  최종 테스트 세트 크기: {len(test_df_keras)}")
print(f"  훈련 세트 라벨 분포:\n{train_df_keras['parsed_label'].value_counts()}")
print(f"  검증 세트 라벨 분포:\n{val_df_keras['parsed_label'].value_counts()}")
print(f"  테스트 세트 라벨 분포:\n{test_df_keras['parsed_label'].value_counts()}")


# Keras flow_from_dataframe을 위한 'filepath'와 'class' 컬럼 생성 함수
def get_spectrogram_filepath_for_keras(row, base_spectrogram_dir):
    base_filename_wav = os.path.basename(os.path.normpath(row['file_name']))
    base_filename_png = base_filename_wav.replace('.wav', '.png')
    
    subset_folder = row['parsed_subset']
    label_folder = row['parsed_label']
    
    return os.path.join(base_spectrogram_dir, subset_folder, label_folder, base_filename_png)

train_df_keras['filepath'] = train_df_keras.apply(lambda row: get_spectrogram_filepath_for_keras(row, output_spectrogram_dir), axis=1)
train_df_keras['class'] = train_df_keras['parsed_label']

val_df_keras['filepath'] = val_df_keras.apply(lambda row: get_spectrogram_filepath_for_keras(row, output_spectrogram_dir), axis=1)
val_df_keras['class'] = val_df_keras['parsed_label']

test_df_keras['filepath'] = test_df_keras.apply(lambda row: get_spectrogram_filepath_for_keras(row, output_spectrogram_dir), axis=1)
test_df_keras['class'] = test_df_keras['parsed_label']


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

keras_class_names = ['anomaly', 'normal']

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df_keras,
    x_col='filepath',
    y_col='class',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    classes=keras_class_names, # 클래스 라벨 명시
    shuffle=True
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df_keras,
    x_col='filepath',
    y_col='class',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    classes=keras_class_names,
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df_keras,
    x_col='filepath',
    y_col='class',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    classes=keras_class_names,
    shuffle=False
)

print("\n--- ImageDataGenerator 설정 완료 ---")
print(f"훈련 제너레이터 클래스 인덱스: {train_generator.class_indices}")
print(f"검증 제너레이터 클래스 인덱스: {validation_generator.class_indices}")
print(f"테스트 제너레이터 클래스 인덱스: {test_generator.class_indices}")


# --- 5. Keras 모델 구축 및 학습 ---
def build_keras_model(model_name, input_shape, num_classes): # num_classes는 1 (이진 분류)
    model = Sequential()

    if model_name == 'custom_cnn':
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))

    elif model_name == 'resnet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
        model.add(base_model)
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        base_model.trainable = False

    elif model_name == 'efficientnetb0':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
        model.add(base_model)
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        base_model.trainable = False
    else:
        raise ValueError(f"Unknown model name: {model_name}")
    
    return model

def train_and_evaluate_keras_model(model, train_gen, val_gen, test_gen, model_name, epochs=10, class_weight=None):
    print(f"\n--- {model_name} 학습 시작 ---")
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    model.summary()

    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
    
    history = model.fit(
        train_gen,
        epochs=epochs,
        validation_data=val_gen,
        callbacks=callbacks,
        class_weight=class_weight # 🐞 class_weight passed to model.fit()
    )
    
    print(f"\n--- {model_name} 학습 완료 ---")

    print(f"\n--- {model_name} 테스트 셋 평가 ---")
    loss, accuracy = model.evaluate(test_gen)
    print(f'Test Loss: {loss:.4f}')
    print(f'Test Accuracy: {accuracy:.4f}')
    
    return model, history

epochs_to_train = 10

# 클래스 가중치 계산 (불균형 해소)
# 훈련 세트 라벨 분포: normal 660, anomaly 60 (이전 출력 기준)
# 훈련 세트 라벨 분포를 value_counts() 결과에서 직접 가져옵니다.
train_label_counts = train_df_keras['parsed_label'].value_counts()
total_train_samples = len(train_df_keras)

# 🐞 get() 메서드를 사용하여 키가 없을 때 KeyError 대신 0을 반환하도록 수정
num_normal = train_label_counts.get('normal', 0)
num_anomaly = train_label_counts.get('anomaly', 0)

# 0으로 나누는 오류 방지
class_weights = {}
if num_anomaly > 0:
    class_weights[train_generator.class_indices['anomaly']] = total_train_samples / (2 * num_anomaly)
else:
    # anomaly 샘플이 훈련 세트에 없는 경우 (매우 드물겠지만)
    # anomaly 클래스에 무한대 가중치를 주거나, 해당 클래스 가중치를 생략할 수 있습니다.
    # 여기서는 계산에 포함하지 않고, 경고 메시지를 출력합니다.
    print("경고: 훈련 세트에 'anomaly' 샘플이 없어 'anomaly' 클래스 가중치를 계산할 수 없습니다.")
    # 이 경우 class_weights 딕셔너리는 normal에 대한 가중치만 가질 것입니다.
    # 모델은 여전히 anomaly를 학습하지 못할 수 있습니다.
    
if num_normal > 0:
    class_weights[train_generator.class_indices['normal']] = total_train_samples / (2 * num_normal)
else:
    print("경고: 훈련 세트에 'normal' 샘플이 없어 'normal' 클래스 가중치를 계산할 수 없습니다.")

print(f"\n계산된 클래스 가중치: {class_weights}")


print("===== Custom CNN 모델 학습 시작 =====")
custom_cnn_model = build_keras_model('custom_cnn', input_shape, 1)
trained_custom_cnn_model, history_cnn = train_and_evaluate_keras_model(
    custom_cnn_model, train_generator, validation_generator, test_generator, 
    "Custom CNN", epochs=epochs_to_train, class_weight=class_weights
)

print("\n===== ResNet50 모델 학습 시작 =====")
resnet50_model = build_keras_model('resnet50', input_shape, 1)
trained_resnet50_model, history_resnet50 = train_and_evaluate_keras_model(
    resnet50_model, train_generator, validation_generator, test_generator, 
    "ResNet50", epochs=epochs_to_train, class_weight=class_weights
)

print("\n===== EfficientNetB0 모델 학습 시작 =====")
efficientnetb0_model = build_keras_model('efficientnetb0', input_shape, 1)
trained_efficientnetb0_model, history_efficientnetb0 = train_and_evaluate_keras_model(
    efficientnetb0_model, train_generator, validation_generator, test_generator, 
    "EfficientNetB0", epochs=epochs_to_train, class_weight=class_weights
)

print("\n모든 Keras 모델 학습 완료!")


설정된 기본 데이터 경로 (원본 WAV): C:\Users\jh\Documents\GitHub\BearingGuardian\Dataset\bearing-raw-mp3
설정된 스펙트로그램 저장 경로: C:\Users\jh\Documents\GitHub\BearingGuardian\Dataset\spectrograms_from_wav

--- WAV 기반 메타데이터 (df_meta.head()) ---
                                           file_name  \
0  bearing/test/section_00_source_test_anomaly_00...   
1  bearing/test/section_00_source_test_anomaly_00...   
2  bearing/test/section_00_source_test_anomaly_00...   
3  bearing/test/section_00_source_test_anomaly_00...   
4  bearing/test/section_00_source_test_anomaly_00...   

                                   relative_wav_path parsed_label  \
0  test/section_00_source_test_anomaly_0000_noAtt...      anomaly   
1  test/section_00_source_test_anomaly_0001_noAtt...      anomaly   
2  test/section_00_source_test_anomaly_0002_noAtt...      anomaly   
3  test/section_00_source_test_anomaly_0003_noAtt...      anomaly   
4  test/section_00_source_test_anomaly_0004_noAtt...      anomaly   

  parsed_subset machine

Generating Spectrograms: 100%|██████████| 1200/1200 [00:32<00:00, 36.98it/s]


--- WAV 기반 스펙트로그램 생성 완료 ---

새로운 데이터 분할 결과:
  최종 훈련 세트 크기: 720
  최종 검증 세트 크기: 240
  최종 테스트 세트 크기: 240
  훈련 세트 라벨 분포:
parsed_label
normal     660
anomaly     60
Name: count, dtype: int64
  검증 세트 라벨 분포:
parsed_label
normal     220
anomaly     20
Name: count, dtype: int64
  테스트 세트 라벨 분포:
parsed_label
normal     220
anomaly     20
Name: count, dtype: int64
Found 720 validated image filenames belonging to 2 classes.
Found 240 validated image filenames belonging to 2 classes.
Found 240 validated image filenames belonging to 2 classes.

--- ImageDataGenerator 설정 완료 ---
훈련 제너레이터 클래스 인덱스: {'anomaly': 0, 'normal': 1}
검증 제너레이터 클래스 인덱스: {'anomaly': 0, 'normal': 1}
테스트 제너레이터 클래스 인덱스: {'anomaly': 0, 'normal': 1}

계산된 클래스 가중치: {0: np.float64(6.0), 1: np.float64(0.5454545454545454)}
===== Custom CNN 모델 학습 시작 =====

--- Custom CNN 학습 시작 ---


c:\Users\jh\.conda\envs\tf-gpu\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 4, 10, 1280)    │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 51200)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │     6,553,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,603,428 (40.45 MB)

 Trainable params: 6,553,857 (25.00 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 20s 619ms/step - accuracy: 0.6100 - loss: 5.0855 - val_accuracy: 0.0833 - val_loss: 0.7239
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 534ms/step - accuracy: 0.4411 - loss: 1.1549 - val_accuracy: 0.9167 - val_loss: 0.6928
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 526ms/step - accuracy: 0.9196 - loss: 0.7054 - val_accuracy: 0.9167 - val_loss: 0.6924
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 519ms/step - accuracy: 0.9102 - loss: 0.7177 - val_accuracy: 0.9167 - val_loss: 0.6930
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 514ms/step - accuracy: 0.9091 - loss: 0.7287 - val_accuracy: 0.9167 - val_loss: 0.6928
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 511ms/step - accuracy: 0.9126 - loss: 0.7085 - val_accuracy: 0.9167 - val_loss: 0.6929

--- EfficientNetB0 학습 완료 ---

--- EfficientNetB0 테스트 셋 평가 ---
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 287ms/step - accuracy: 0.9222 - loss: 0.6924
Test Loss: 0.6924
Test Accuracy: 0.9167

모든 Keras 모델 학습 완료!


In [10]:
import os
import pandas as pd
import numpy as np # 넘파이는 스펙트로그램 생성 시 필요하지만, 로드 단계에서도 임포트 유지
from sklearn.model_selection import train_test_split
from PIL import Image # For Keras image loading

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score, accuracy_score

# --- 경로 및 파라미터 재설정 (스펙트로그램 생성 단계에서 사용된 것과 동일) ---
# base_data_path는 원본 .wav 파일의 경로 (이제 직접 로드하지 않으므로, 정확성만 확인)
base_data_path = r'C:\Users\jh\Documents\GitHub\BearingGuardian\Dataset\bearing-raw-mp3' 
# output_spectrogram_dir는 스펙트로그램 .png 파일이 저장된 경로
output_spectrogram_dir = os.path.join(os.path.dirname(base_data_path), 'spectrograms_from_wav')

# 메타데이터 파일 경로 (이전에 사용된 attributes_00.csv 경로)
meta_file_path = os.path.join(os.path.dirname(base_data_path), 'bearing', 'attributes_00.csv')

# 스펙트로그램 생성 파라미터 (이미지 크기 계산에 필요)
sr_target = 16000
n_fft = 1024
hop_length = 512
n_mels = 128

# 이미지 크기 및 채널 설정
img_height, img_width = n_mels, int(sr_target * 10 / hop_length) 
input_shape = (img_height, img_width, 3) # Keras 모델 입력 형태: (높이, 너비, 채널) -> RGB 3채널

batch_size = 32
num_classes = 2 # normal, anomaly (이진 분류)

print(f"스펙트로그램 이미지 입력 형태: {input_shape}")
print(f"배치 크기: {batch_size}")
print(f"분류할 클래스 수: {num_classes}")

# --- 메타데이터 로드 및 파싱 (이전에 완료된 작업이지만, 변수 로드를 위해 다시 실행) ---
try:
    df_meta_original = pd.read_csv(meta_file_path)
    df_meta_original['relative_wav_path'] = df_meta_original['file_name'].apply(lambda x: x.replace('bearing/', '', 1))
    df_meta_original['parsed_label'] = df_meta_original['relative_wav_path'].apply(lambda x: 'normal' if 'normal' in x else ('anomaly' if 'anomaly' in x else 'unknown'))
    df_meta_original['parsed_subset'] = df_meta_original['relative_wav_path'].apply(lambda x: 'train' if 'train' in x else ('test' if 'test' in x else 'unknown'))
    df_meta_original['machine_id'] = 'bearing'
    df_meta = df_meta_original
    
    print("\n--- 메타데이터 로드 완료 (df_meta.head()) ---")
    print(df_meta.head())

except FileNotFoundError:
    print(f"오류: 원본 메타데이터 파일을 찾을 수 없습니다: {meta_file_path}. 이전에 스펙트로그램 생성 단계가 완료되었는지 확인해주세요.")
    exit()
except Exception as e:
    print(f"메타데이터 로드 및 파싱 중 오류 발생: {e}")
    exit()

# --- 데이터 분할 (스펙트로그램 생성 시와 동일하게) ---
train_val_combined_df, test_df_keras = train_test_split(
    df_meta, test_size=0.2, random_state=42, stratify=df_meta['parsed_label']
)
train_df_keras, val_df_keras = train_test_split(
    train_val_combined_df, test_size=0.25, random_state=42, stratify=train_val_combined_df['parsed_label']
)

print(f"\n새로운 데이터 분할 결과:")
print(f"  최종 훈련 세트 크기: {len(train_df_keras)}")
print(f"  최종 검증 세트 크기: {len(val_df_keras)}")
print(f"  최종 테스트 세트 크기: {len(test_df_keras)}")
print(f"  훈련 세트 라벨 분포:\n{train_df_keras['parsed_label'].value_counts()}")
print(f"  검증 세트 라벨 분포:\n{val_df_keras['parsed_label'].value_counts()}")
print(f"  테스트 세트 라벨 분포:\n{test_df_keras['parsed_label'].value_counts()}")

# --- Keras flow_from_dataframe을 위한 'filepath'와 'class' 컬럼 생성 ---
def get_spectrogram_filepath_for_keras(row, base_spectrogram_dir):
    base_filename_wav = os.path.basename(os.path.normpath(row['file_name']))
    base_filename_png = base_filename_wav.replace('.wav', '.png')
    
    subset_folder = row['parsed_subset'] # 원본 DCASE의 train/test 폴더명
    label_folder = row['parsed_label']   # normal/anomaly 폴더명
    
    return os.path.join(base_spectrogram_dir, subset_folder, label_folder, base_filename_png)

train_df_keras['filepath'] = train_df_keras.apply(lambda row: get_spectrogram_filepath_for_keras(row, output_spectrogram_dir), axis=1)
train_df_keras['class'] = train_df_keras['parsed_label']

val_df_keras['filepath'] = val_df_keras.apply(lambda row: get_spectrogram_filepath_for_keras(row, output_spectrogram_dir), axis=1)
val_df_keras['class'] = val_df_keras['parsed_label']

test_df_keras['filepath'] = test_df_keras.apply(lambda row: get_spectrogram_filepath_for_keras(row, output_spectrogram_dir), axis=1)
test_df_keras['class'] = test_df_keras['parsed_label']


# --- ImageDataGenerator 설정 (데이터 증강 강화) ---
train_datagen = ImageDataGenerator(
    rescale=1./255, # 픽셀 값 0-1 스케일링
    rotation_range=20, # 20도 내에서 무작위 회전
    width_shift_range=0.2, # 가로 이동 (전체 너비의 20% 내)
    height_shift_range=0.2, # 세로 이동 (전체 높이의 20% 내)
    zoom_range=0.2, # 줌 범위 (20% 확대/축소)
    brightness_range=[0.8, 1.2], # 밝기 범위 (80%~120%)
    horizontal_flip=True, # 무작위 수평 뒤집기
    fill_mode='nearest' # 비어있는 픽셀 채우기
)

val_datagen = ImageDataGenerator(rescale=1./255) # 검증 데이터는 증강 없음
test_datagen = ImageDataGenerator(rescale=1./255) # 테스트 데이터는 증강 없음

keras_class_names = ['anomaly', 'normal'] # 클래스 순서 (알파벳 순서에 따름)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df_keras,
    x_col='filepath',
    y_col='class',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary', # 이진 분류 (출력 뉴런 1개)
    classes=keras_class_names, # 클래스 라벨 명시
    shuffle=True
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df_keras,
    x_col='filepath',
    y_col='class',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    classes=keras_class_names,
    shuffle=False # 검증 셋은 섞지 않음
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df_keras,
    x_col='filepath',
    y_col='class',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    classes=keras_class_names,
    shuffle=False # 테스트 셋은 섞지 않음
)

print("\n--- ImageDataGenerator 설정 완료 ---")
print(f"훈련 제너레이터 클래스 인덱스: {train_generator.class_indices}")
print(f"검증 제너레이터 클래스 인덱스: {validation_generator.class_indices}")
print(f"테스트 제너레이터 클래스 인덱스: {test_generator.class_indices}")



스펙트로그램 이미지 입력 형태: (128, 312, 3)
배치 크기: 32
분류할 클래스 수: 2

--- 메타데이터 로드 완료 (df_meta.head()) ---
                                           file_name  \
0  bearing/test/section_00_source_test_anomaly_00...   
1  bearing/test/section_00_source_test_anomaly_00...   
2  bearing/test/section_00_source_test_anomaly_00...   
3  bearing/test/section_00_source_test_anomaly_00...   
4  bearing/test/section_00_source_test_anomaly_00...   

                                   relative_wav_path parsed_label  \
0  test/section_00_source_test_anomaly_0000_noAtt...      anomaly   
1  test/section_00_source_test_anomaly_0001_noAtt...      anomaly   
2  test/section_00_source_test_anomaly_0002_noAtt...      anomaly   
3  test/section_00_source_test_anomaly_0003_noAtt...      anomaly   
4  test/section_00_source_test_anomaly_0004_noAtt...      anomaly   

  parsed_subset machine_id  
0          test    bearing  
1          test    bearing  
2          test    bearing  
3          test    bearing  
4         

In [ ]:
# --- Keras 모델 구축 함수 정의 ---
def build_keras_model(model_name, input_shape, num_classes): # num_classes는 1 (이진 분류)
    model = Sequential()

    if model_name == 'custom_cnn':
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid')) # 이진 분류이므로 출력 뉴런 1개, sigmoid 활성화 함수

    elif model_name == 'resnet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
        model.add(base_model)
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        base_model.trainable = False

    elif model_name == 'efficientnetb0':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
        model.add(base_model)
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        base_model.trainable = False
    else:
        raise ValueError(f"Unknown model name: {model_name}")
    
    return model

# --- 모델 학습 및 평가 함수 정의 ---
def train_and_evaluate_keras_model(model, train_gen, val_gen, test_gen, model_name, epochs=10, class_weight=None):
    print(f"\n--- {model_name} 학습 시작 ---")
    
    # 모델 컴파일: 학습률 (Learning Rate) 감소
    model.compile(optimizer=Adam(learning_rate=0.0001), # 0.001 -> 0.0001로 10배 감소
                  loss='binary_crossentropy', # 이진 분류 손실 함수
                  metrics=['accuracy']) # 정확도 지표 사용
    
    model.summary() # 모델 구조 요약 출력

    # 콜백 (Early Stopping): patience 증가 (3 -> 5)
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)] 
    
    # 모델 학습
    history = model.fit(
        train_gen, # 훈련 데이터 제너레이터
        epochs=epochs, # 학습 에포크 수
        validation_data=val_gen, # 검증 데이터 제너레이터
        callbacks=callbacks, # 콜백 적용
        class_weight=class_weight # 클래스 가중치 적용
    )
    
    print(f"\n--- {model_name} 학습 완료 ---")

    # 모델 평가 (테스트 셋)
    print(f"\n--- {model_name} 테스트 셋 평가 ---")
    loss, accuracy = model.evaluate(test_gen) # 테스트 셋으로 손실과 정확도 계산
    print(f'Test Loss: {loss:.4f}')
    print(f'Test Accuracy: {accuracy:.4f}')
    
    # 상세 평가 지표 추가
    print(f"\n--- {model_name} 상세 평가 지표 ---")
    
    test_gen.reset() # 제너레이터 초기화
    y_pred_probs = model.predict(test_gen) # 테스트 제너레이터에서 예측 수행
    y_pred = (y_pred_probs > 0.5).astype(int) # sigmoid 출력 (0~1)을 0 또는 1로 변환
    
    y_true = test_gen.classes # 실제 라벨 가져오기 (제너레이터의 라벨 순서와 매칭)
    
    # 클래스 이름 (anomaly:0, normal:1)
    target_names = [k for k, v in sorted(test_gen.class_indices.items(), key=lambda item: item[1])]
    
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=target_names))
    
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    
    # F1-Score, Precision, Recall은 classification_report에 포함되지만 개별적으로도 출력
    print(f"F1-Score (weighted): {f1_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"Precision (weighted): {precision_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"Recall (weighted): {recall_score(y_true, y_pred, average='weighted'):.4f}")
    
    return model, history # 학습된 모델과 학습 이력 반환

# --- 각 모델별 학습 실행 ---
epochs_to_train = 20 # 에포크 수 증가 (10 -> 20)

# 클래스 가중치 계산 (불균형 해소)
train_label_counts = train_df_keras['parsed_label'].value_counts()
total_train_samples = len(train_df_keras)

num_normal = train_label_counts.get('normal', 0)
num_anomaly = train_label_counts.get('anomaly', 0)

class_weights = {}
# 'anomaly'와 'normal' 모두 샘플이 0개가 아닌 경우에만 가중치 계산
if num_anomaly > 0 and num_normal > 0:
    class_weights[train_generator.class_indices['anomaly']] = total_train_samples / (2 * num_anomaly)
    class_weights[train_generator.class_indices['normal']] = total_train_samples / (2 * num_normal)
elif num_anomaly == 0:
    print("경고: 훈련 세트에 'anomaly' 샘플이 없어 'anomaly' 클래스 가중치를 계산할 수 없습니다. 클래스 불균형이 심각합니다.")
    # 이 경우 'normal'에 대한 가중치만 계산되거나, 가중치 적용이 무의미해질 수 있습니다.
    # 모델 학습에 큰 영향을 줄 수 있으므로, anomaly 샘플 확보가 중요합니다.
    if num_normal > 0:
        class_weights[train_generator.class_indices['normal']] = total_train_samples / (2 * num_normal)
elif num_normal == 0:
    print("경고: 훈련 세트에 'normal' 샘플이 없어 'normal' 클래스 가중치를 계산할 수 없습니다.")
    if num_anomaly > 0:
        class_weights[train_generator.class_indices['anomaly']] = total_train_samples / (2 * num_anomaly)


print(f"\n계산된 클래스 가중치: {class_weights}")


print("===== Custom CNN 모델 학습 시작 =====")
custom_cnn_model = build_keras_model('custom_cnn', input_shape, 1) # num_classes = 1 for binary
trained_custom_cnn_model, history_cnn = train_and_evaluate_keras_model(
    custom_cnn_model, train_generator, validation_generator, test_generator, 
    "Custom CNN", epochs=epochs_to_train, class_weight=class_weights # 클래스 가중치 적용
)

print("\n===== ResNet50 모델 학습 시작 =====")
resnet50_model = build_keras_model('resnet50', input_shape, 1) # num_classes = 1 for binary
trained_resnet50_model, history_resnet50 = train_and_evaluate_keras_model(
    resnet50_model, train_generator, validation_generator, test_generator, 
    "ResNet50", epochs=epochs_to_train, class_weight=class_weights # 클래스 가중치 적용
)

print("\n===== EfficientNetB0 모델 학습 시작 =====")
efficientnetb0_model = build_keras_model('efficientnetb0', input_shape, 1) # num_classes = 1 for binary
trained_efficientnetb0_model, history_efficientnetb0 = train_and_evaluate_keras_model(
    efficientnetb0_model, train_generator, validation_generator, test_generator, 
    "EfficientNetB0", epochs=epochs_to_train, class_weight=class_weights # 클래스 가중치 적용
)

print("\n모든 Keras 모델 학습 완료!")